**AMSfCA**

Computer Vision lab, checkerboard tracking

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os

Import images

In [ ]:
# Try to import one image
image = cv.imread("/content/01.png")
print(image.shape)
plt.imshow(image)
plt.axis("off")
plt.title("our first image")
plt.show()

In [ ]:
# Get the path of each image
images_paths = glob.glob("/content/*.png")
print(images_paths)
print(f'the number of pictures is {len(images_paths)}')

In [ ]:
# Import images
for k, path in enumerate(images_paths):
  image = cv.imread(path)
  print(image.shape)
  #plt.imshow(image)
  #plt.axis("off")
  #plt.title(f'image number {k}')
  #plt.show()

Camera calibration

In [ ]:
# Define the size of the checkerboard
CHECKERBOARD = (8,5)  # corners in the checkerboard
squareDim = 15.0      # length of each square in the checkerboard [mm]

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = []
for i in range(CHECKERBOARD[1]):
  for j in range(CHECKERBOARD[0]):
    objp.append([j*squareDim, i*squareDim, 0.0])

objp = np.array(objp, np.float32)

object_points = []
images_points = []


In [ ]:
for k, path in enumerate(images_paths):
  image = cv.imread(path) # import image
  print(image.shape)
  gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)  # convert RGB image in gray
  print(gray.shape)

  # Find chessboard corners
  ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)
  print(ret, len(corners))

  if ret == True:
    # Look at sub-pixel level to refine the detected position of the corners
    corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

    # Save positions of the corners
    images_points.append(corners2)
    object_points.append(objp)

    image = cv.drawChessboardCorners(image, CHECKERBOARD, corners2, True)

    plt.imshow(image)
    plt.axis("off")
    plt.show()


In [ ]:
print(object_points)

In [ ]:
print(images_points)

In [ ]:
# Calibrate camera
ret, intrinsic, dist, r_vecs, t_vecs = cv.calibrateCamera(object_points, images_points, (1216,1936), None, None)

print(f"Overall RMS re-projection error: {ret}")
print("\nCamera (intrinsic) matrix:")
print(intrinsic)
print("\nDistortion coefficients:")
print(dist)
print("\nRotation vectors estimated for each pattern view:")
print(r_vecs)
print("\nTranslation vectors estimated for each pattern view:")
print(t_vecs)

Analyse a new video and track the calibrator

In [ ]:
# This function takes the corners in the chessboard and axis points to draw a 3D axis
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel().astype("int32"))
    imgpts = imgpts.astype("int32")
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [ ]:
# This funciton merges images to create a video
def convert_images_to_video(image_files, output_file, fps):

    # Read the first image to get its dimensions
    first_image = image_files[0]
    height, width, _ = first_image.shape

    # Create a VideoWriter object to save the video
    fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Specify the codec for the output video file
    video = cv.VideoWriter(output_file, fourcc, fps, (width, height))

    # Iterate over each image and write it to the video
    for frame in image_files:
        video.write(frame)

    # Release the video writer and close the video file
    video.release()
    cv.destroyAllWindows()

In [ ]:
# Import video to analyse
video_path = "/content/video.avi"
cap = cv.VideoCapture(video_path)

t_vec_history = []
r_vec_history = []

t_x_history = []
t_y_history = []
t_z_history = []

img_frames = []

# Define the 3 axes that will be printed
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

In [ ]:
# Open and analyse each frame of the video

while cap.isOpened():
  ret, frame = cap.read()

  if not ret:
    break

  gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)  # Convert each colour frame in gray
  ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)
  if ret == True:
    corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

    # Return the rotation and translation vectors that transform a 3D point expressed in the object coordinate frame to the camera coordinate frame
    ret, r_vec, t_vec = cv.solvePnP(objp, corners2, intrinsic, dist)

    # project 3D points to image plane (imgpts are the pixel coordinates of the end of each axis that we have defined before in 3D)
    imgpts, jac = cv.projectPoints(axis, r_vec, t_vec, intrinsic, dist)

    # Call the function to draw 3D axis on the chessboard
    img = draw(frame,corners2,imgpts)

    # Store each frame with the plot of the 3D axis in a list that will be used to create the video
    img_frames.append(img)

    # Put translation vectors in variables to store the history of the movement
    t_vec_history.append(t_vec)
    t_x_history.append(t_vec[0])
    t_y_history.append(t_vec[1])
    t_z_history.append(t_vec[2])

    # Put rotation vectors in variables to store the history of the movement
    r_vec_history.append(r_vec)

# Call the function to convert the images to video
output_file = "/content/video_output.mp4"
convert_images_to_video(img_frames, output_file, fps=10)



Plot movements

In [ ]:
# Plot the translation over time
plt.plot(t_x_history, label="X", color = 'g')
plt.plot(t_y_history, label="Y", color = 'b')
plt.plot(t_z_history, label="Z", color = 'r')
plt.legend()
plt.xlabel("Time [s]")
plt.ylabel("Translation [mm]")
plt.title("Movement of the checkerboard wrt the optical centre of the camera")
plt.show()

In [ ]:
x = np.array(t_x_history).reshape(-1)
y = np.array(t_y_history).reshape(-1)
z = np.array(t_z_history).reshape(-1)

In [ ]:
# Show the translation in a 3D space

import plotly.graph_objects as go
import plotly

fig = go.Figure(data=[
    go.Scatter3d(x=x, y=y, z=z, mode="markers", name="Chessboard positions")
])

# Add a fixed point to represent the camera in the origin
fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode="markers+text",
    marker=dict(size=8, color='red'),
    text=["Camera"],
    textposition="top center",
    name="Camera origin"
))

fig.update_layout(
    title="Chessboard positions in camera frame",
    scene=dict(
        xaxis_title="X (mm)",
        yaxis_title="Y (mm)",
        zaxis_title="Z (mm)",
        aspectmode='data'
    )
)

fig.show()

In [ ]:
camera_positions = []
camera_directions = []


for r_vec, t_vec in zip(r_vec_history, t_vec_history):
    # Convert r_vec in rotation matrix
    R, _ = cv.Rodrigues(r_vec)

    # Invert the transformation: compute the position of the camera in the chessboard reference system
    R_inv = R.T
    t_inv = -R_inv @ t_vec
    cam_pos = t_inv.flatten()

    camera_positions.append(t_inv.flatten())

    # Camera Z-axis in chessboard frame is the third column if R_inv.
    # Let's multiply for a scale for better visualization
    z_axis = R_inv[:, 2] * 50
    camera_directions.append((cam_pos, cam_pos + z_axis))

# Get X, Y, Z coordinates from camera
x = [pos[0] for pos in camera_positions]
y = [pos[1] for pos in camera_positions]
z = [pos[2] for pos in camera_positions]


# Visualize camera position in 3D space
fig = go.Figure(data=[
    go.Scatter3d(x=x, y=y, z=z, mode="markers", name="Camera positions")
])

# Add a fixed point to represent the chessboard in the origin
fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode="markers+text",
    marker=dict(size=8, color='red'),
    text=["Chessboard"],
    textposition="top center",
    name="Chessboard origin"
))

# Draw arrows for camera z-axis (direction of camera view)
for start, end in camera_directions:
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode="lines",
        line=dict(color="green", width=4),
        showlegend=False
    ))


fig.update_layout(
    title="Camera positions and directions in chessboard frame",
    scene=dict(
        xaxis_title="X (mm)",
        yaxis_title="Y (mm)",
        zaxis_title="Z (mm)",
        aspectmode='data'
    )
)

fig.show()
